In [22]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import yfinance as yf
import pandas_market_calendars as mcal
from plotly.offline import init_notebook_mode, plot
init_notebook_mode(connected=True)



In [23]:
portfolio_df = pd.read_csv(r'C:\Users\socce\Documents\mpstocks1.csv')
portfolio_df['Open date'] = pd.to_datetime(portfolio_df['Open date'])
portfolio_df['Qty'] = abs(portfolio_df['Qty']) 
portfolio_df['Adj cost'] = abs(portfolio_df['Adj cost']) 
portfolio_df['Adj cost per share'] = abs(portfolio_df['Adj cost per share']) 
portfolio_df = portfolio_df[(portfolio_df['Type'] == 'Bought') | (portfolio_df['Type'] == 'Sold')]
# portfolio_df.loc[-1] = ['ADXS', 'A', 125, 'Bought', '2018-01-5', 31.62, 31.62 *125]  # adding a row
# portfolio_df.index = portfolio_df.index + 1  # shifting index
# portfolio_df = portfolio_df.sort_index()  # sorting by index
#portfolio_df = portfolio_df[(portfolio_df['Symbol'] == 'FB')]# | (portfolio_df['Symbol'] == 'FB')]

In [24]:
portfolio_df

,Symbol,Security,Qty,Type,Open date,Adj cost per share,Adj cost
0,AAPL,A,25.0,Bought,2017-01-13,119.1300,2986.24
1,AAPL,A,25.0,Bought,2020-03-03,293.4800,7337.00
2,AAPL,A,378.0,Bought,2020-08-31,0.0000,0.00
3,AAPL,A,51.0,Bought,2020-08-31,127.9300,6524.43
4,ACWX,A,50.0,Bought,2017-12-27,49.7374,2491.82
...,...,...,...,...,...,...,...
136,VXRT,A,500.0,Bought,2020-11-02,4.9800,2490.00
137,XBI,A,25.0,Bought,2016-01-11,57.5875,1447.68
138,XLI,A,35.0,Bought,2020-12-21,88.1100,3083.85
139,XLK,A,50.0,Bought,2017-12-27,64.1674,3213.32


In [34]:
symbols = portfolio_df.Symbol.unique()

start_date = datetime.datetime(2014, 6, 8)
stocks_end = datetime.datetime.today() #datetime.datetime(2021, 1, 1)


#calendar of all dates that the market was open for the specified date range
def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(start_date, stocks_end) # start_date used to be stocks_start
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal

#returns concatenated dataframe of all stock info for each company wihtin the specified date range (whatever is availble)
def get_data(stocks, start, end):
    def data(ticker):
        df = yf.download(ticker, start=start, end=(end + datetime.timedelta(days=1)))
        df['symbol'] = ticker
        df.index = pd.to_datetime(df.index)
        return df
    datas = map(data, stocks)
    return(pd.concat(datas, keys=stocks, names=['Ticker', 'Date'], sort=True))


#uses get_data function to get all the information for the SPDR S&P 500 ETF Trust
def get_benchmark(benchmark, start, end):
    benchmark = get_data(benchmark, start, end)
    benchmark = benchmark.drop(['symbol'], axis=1)
    benchmark.reset_index(inplace=True)
    return benchmark

daily_adj_close = get_data(symbols, start_date, stocks_end) #stocks_start
daily_adj_close = daily_adj_close[['Close']].reset_index()
daily_benchmark = get_benchmark(['SPY'], start_date, stocks_end) #stocks_start
daily_benchmark = daily_benchmark[['Date', 'Close']]
market_cal = create_market_cal(start_date, stocks_end) #stocks_start



def position_adjust(daily_positions, sale):
    stocks_with_sales = pd.DataFrame()
    buys_before_start = daily_positions[daily_positions['Type'] == 'Bought'].sort_values(by='Open date')
    for position in buys_before_start[buys_before_start['Symbol'] == sale[1]['Symbol']].iterrows():
        if position[1]['Qty'] <= sale[1]['Qty']:
            sale[1]['Qty'] -= position[1]['Qty']
            position[1]['Qty'] = 0
        else:
            position[1]['Qty'] -= sale[1]['Qty']
            sale[1]['Qty'] -= sale[1]['Qty']
        stocks_with_sales = stocks_with_sales.append(position[1])
    return stocks_with_sales



def portfolio_start_balance(portfolio, start_date):
    positions_before_start = portfolio[portfolio['Open date'] <= stocks_end]#stocks_start]#start_date]
    future_positions = portfolio[portfolio['Open date'] >= start_date]
    sales = positions_before_start[positions_before_start['Type'] == 'Sold'].groupby(['Symbol'])['Qty'].sum()
    sales = sales.reset_index()
    positions_no_change = positions_before_start[~positions_before_start['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions_df = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = position_adjust(positions_before_start, sale)
        adj_positions_df = adj_positions_df.append(adj_positions)
    adj_positions_df = adj_positions_df.append(positions_no_change)
    adj_positions_df = adj_positions_df.append(future_positions)
    adj_positions_df = adj_positions_df[adj_positions_df['Qty'] > 0]
    return adj_positions_df


active_portfolio = portfolio_start_balance(portfolio_df, stocks_end)#stocks_start)
active_portfolio = active_portfolio.merge(daily_adj_close, left_on = ['Symbol','Open date'], right_on = ['Ticker','Date'])
active_portfolio = active_portfolio[['Symbol','Security','Qty','Type','Open date','Close']]
active_portfolio['Adj cost'] = active_portfolio['Close'] * active_portfolio['Qty']
active_portfolio = active_portfolio.rename({'Close':'Adj cost per share'}, axis = 1)


def fifo(daily_positions, sales, date):
    sales = sales[sales['Open date'] == date]
    daily_positions = daily_positions[daily_positions['Open date'] <= date]
    positions_no_change = daily_positions[~daily_positions['Symbol'].isin(sales['Symbol'].unique())]
    adj_positions = pd.DataFrame()
    for sale in sales.iterrows():
        adj_positions = adj_positions.append(position_adjust(daily_positions, sale))
    adj_positions = adj_positions.append(positions_no_change)
    adj_positions = adj_positions[adj_positions['Qty'] > 0]
    return adj_positions


def time_fill(portfolio, market_cal):
    sales = portfolio[portfolio['Type'] == 'Sold'].groupby(['Symbol','Open date'])['Qty'].sum()
    sales = sales.reset_index()
    per_day_balance = []
    for date in market_cal:
        if (sales['Open date'] == date).any():
            portfolio = fifo(portfolio, sales, date)
        daily_positions = portfolio[portfolio['Open date'] <= date]
        daily_positions = daily_positions[daily_positions['Type'] == 'Bought']
        daily_positions['Date Snapshot'] = date
        per_day_balance.append(daily_positions)
    return per_day_balance

# matches prices of each asset to open date, then adjusts for  cps of dates
def modified_cost_per_share(portfolio, adj_close, start_date):
    df = pd.merge(portfolio, adj_close, left_on=['Date Snapshot', 'Symbol'],
                  right_on=['Date', 'Ticker'], how='left')
    df.rename(columns={'Close': 'Symbol Adj Close'}, inplace=True)
    df['Adj cost daily'] = df['Symbol Adj Close'] * df['Qty']
    df = df.drop(['Ticker', 'Date'], axis=1)
    return df


# merge portfolio data with latest benchmark data and create several calcs
def benchmark_portfolio_calcs(portfolio, benchmark):
    portfolio = pd.merge(portfolio, benchmark, left_on=['Date Snapshot'],
                         right_on=['Date'], how='left')
    portfolio = portfolio.drop(['Date'], axis=1)
    portfolio.rename(columns={'Close': 'Benchmark Close'}, inplace=True)
    benchmark_max = benchmark[benchmark['Date'] == portfolio['Date Snapshot'].max()]
    portfolio['Benchmark End Date Close'] = portfolio.apply(lambda x: benchmark_max['Close'], axis=1)
    portfolio = portfolio.merge(benchmark, left_on = 'Open date', right_on = 'Date')
    portfolio = portfolio.drop('Date', axis = 1).rename({'Close':'Benchmark Start Date Close'}, axis = 1)
    return portfolio


def portfolio_end_of_year_stats(portfolio, adj_close_end):
    adj_close_end = adj_close_end[adj_close_end['Date'] == adj_close_end['Date'].max()-datetime.timedelta(days=1)]
    portfolio_end_data = pd.merge(portfolio, adj_close_end, left_on='Symbol',
                                  right_on='Ticker')
    portfolio_end_data.rename(columns={'Close': 'Ticker End Date Close'}, inplace=True)
    portfolio_end_data = portfolio_end_data.drop(['Ticker', 'Date'], axis=1)
    return portfolio_end_data


# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
def portfolio_start_of_year_stats(portfolio, adj_close_start):
    adj_close_start = adj_close_start[adj_close_start['Date'] == adj_close_start['Date'].min()]
    portfolio_start = pd.merge(portfolio, adj_close_start[['Ticker', 'Close', 'Date']],
                                    left_on='Symbol', right_on='Ticker')
    portfolio_start.rename(columns={'Close': 'Ticker Start Date Close'}, inplace=True)
    portfolio_start['Adj cost per share'] = np.where(portfolio_start['Open date'] <= portfolio_start['Date'],
                                                          portfolio_start['Ticker Start Date Close'],
                                                          portfolio_start['Adj cost per share'])
    portfolio_start['Adj cost'] = portfolio_start['Adj cost per share'] * portfolio_start['Qty']
    portfolio_start = portfolio_start.drop(['Ticker', 'Date'], axis=1)
    portfolio_start['Equiv Benchmark Shares'] = portfolio_start['Adj cost'] / portfolio_start['Benchmark Start Date Close'] 
    portfolio_start['Benchmark Start Date Cost'] = portfolio_start['Equiv Benchmark Shares'] * portfolio_start['Benchmark Start Date Close']
    return portfolio_start

def calc_returns(portfolio): 
    portfolio['Benchmark Return'] = portfolio['Benchmark Close'] / portfolio['Benchmark Start Date Close'] - 1
    portfolio['Ticker Return'] = portfolio['Symbol Adj Close'] / portfolio['Adj cost per share'] - 1
    portfolio['Ticker Share Value'] = portfolio['Qty'] * portfolio['Symbol Adj Close']
    portfolio['Benchmark Share Value'] = portfolio['Equiv Benchmark Shares'] * portfolio['Benchmark Close']
    portfolio['Stock Gain / (Loss)'] = portfolio['Ticker Share Value'] - portfolio['Adj cost']
    portfolio['Benchmark Gain / (Loss)'] = portfolio['Benchmark Share Value'] - portfolio['Adj cost']
    portfolio['Abs Value Compare'] = portfolio['Ticker Share Value'] - portfolio['Benchmark Start Date Cost']
    portfolio['Abs Value Return'] = portfolio['Abs Value Compare']/portfolio['Benchmark Start Date Cost']
    portfolio['Abs. Return Compare'] = portfolio['Ticker Return'] - portfolio['Benchmark Return']
    return portfolio


def per_day_portfolio_calcs(per_day_holdings, daily_benchmark, daily_adj_close, start_date): #used to be stocks_Start
    df = pd.concat(per_day_holdings, sort=True)
    mcps = modified_cost_per_share(df, daily_adj_close, start_date)
    mcps['Adj cost per share'] = mcps['Adj cost per share'].replace(to_replace=0, method='ffill')
    bpc = benchmark_portfolio_calcs(mcps, daily_benchmark)
    pes = portfolio_end_of_year_stats(bpc, daily_adj_close)
    pss = portfolio_start_of_year_stats(pes, daily_adj_close)
    returns = calc_returns(pss)
    return returns


# # portfolio_df = pd.read_csv(r'C:\Users\socce\Documents\Book3.csv')
# # portfolio_df['Open date'] = pd.to_datetime(portfolio_df['Open date'])


positions_per_day = time_fill(active_portfolio, market_cal)
combined_df = per_day_portfolio_calcs(positions_per_day, daily_benchmark,
                                      daily_adj_close, start_date) #stocks_start


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [35]:
def line_facets(df, val_1, val_2):
    grouped_metrics = combined_df.groupby(['Symbol','Date Snapshot'])[[val_1,val_2]].sum().reset_index()
    grouped_metrics = pd.melt(grouped_metrics, id_vars=['Symbol','Date Snapshot'],
                              value_vars=[val_1, val_2])
    fig = px.line(grouped_metrics, x="Date Snapshot", y="value",
                facet_col="Symbol", facet_col_wrap=5,color='variable')
    fig.update_yaxes(matches=None)
    fig.update_xaxes(matches=None)
    plot(fig)


def line(df, val_1, val_2):
    grouped_metrics = combined_df.groupby(['Date Snapshot'])[[val_1,val_2]].sum().reset_index()
    grouped_metrics = pd.melt(grouped_metrics, id_vars=['Date Snapshot'],
                              value_vars=[val_1, val_2])
    fig = px.line(grouped_metrics, x="Date Snapshot", y="value", 
                  color='variable')
    plot(fig)

In [36]:
line_facets(combined_df, 'Ticker Return', 'Benchmark Return')

In [37]:
combined_df[['Adj cost', 'Adj cost per share', 'Date Snapshot', 'Open date', 'Qty',
       'Symbol', 'Type', 'Symbol Adj Close', 'Adj cost daily',
       'Benchmark Close', 'Benchmark End Date Close',
       'Benchmark Start Date Close',
        'Equiv Benchmark Shares',
       'Benchmark Start Date Cost', 'Benchmark Return', 'Ticker Return',
       'Ticker Share Value', 'Benchmark Share Value']]


,Adj cost,Adj cost per share,Date Snapshot,Open date,Qty,Symbol,Type,Symbol Adj Close,Adj cost daily,Benchmark Close,Benchmark End Date Close,Benchmark Start Date Close,Equiv Benchmark Shares,Benchmark Start Date Cost,Benchmark Return,Ticker Return,Ticker Share Value,Benchmark Share Value
0,4202.799928,2.80,2018-06-26,2018-06-26,1501.0,AMRN,Bought,2.80,4202.799928,271.600006,382.880005,271.600006,15.474226,4202.799928,0.000000,0.000000,4202.799928,4202.799928
1,4202.799928,2.80,2018-06-27,2018-06-26,1501.0,AMRN,Bought,2.75,4127.750000,269.350006,382.880005,271.600006,15.474226,4202.799928,-0.008284,-0.017857,4127.750000,4167.982919
2,4202.799928,2.80,2018-06-28,2018-06-26,1501.0,AMRN,Bought,3.03,4548.029957,270.890015,382.880005,271.600006,15.474226,4202.799928,-0.002614,0.082143,4548.029957,4191.813360
3,4202.799928,2.80,2018-06-29,2018-06-26,1501.0,AMRN,Bought,3.09,4638.089871,271.279999,382.880005,271.600006,15.474226,4202.799928,-0.001178,0.103571,4638.089871,4197.848063
4,4202.799928,2.80,2018-07-02,2018-06-26,1501.0,AMRN,Bought,2.94,4412.940086,271.859985,382.880005,271.600006,15.474226,4202.799928,0.000957,0.050000,4412.940086,4206.822906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768,2694.999933,0.77,2021-01-15,2020-12-23,3500.0,PTN,Bought,0.78,2729.999900,375.700012,382.880005,367.570007,7.331936,2694.999933,0.022118,0.012987,2729.999900,2754.608612
8769,2694.999933,0.77,2021-01-19,2020-12-23,3500.0,PTN,Bought,0.90,3149.999917,378.649994,382.880005,367.570007,7.331936,2694.999933,0.030144,0.168831,3149.999917,2776.237691
8770,2694.999933,0.77,2021-01-20,2020-12-23,3500.0,PTN,Bought,1.01,3534.999967,383.890015,382.880005,367.570007,7.331936,2694.999933,0.044400,0.311688,3534.999967,2814.657190
8771,2694.999933,0.77,2021-01-21,2020-12-23,3500.0,PTN,Bought,1.15,4024.999917,384.239990,382.880005,367.570007,7.331936,2694.999933,0.045352,0.493506,4024.999917,2817.223188
